# Simple ML - Testing simple models#

What it does?
- Reads the csv file created by the Statistics Parser
- Drops the string columns and keeps only the columns with the grades
- Tests a model for each course-column using the LeaveOneOut method and the XGBoost Regressor
- Prints the results in terms of MAE and RMSE

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import LeaveOneOut
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
import math

import warnings #needed for this type of classifier
warnings.simplefilter(action='ignore', category=Warning)


In [2]:
data = pd.read_csv("csd.csv")
display(data)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:,...,NGE-08-02,NGE-07-03,NGE-06-04,NGE-06-05,NGE-08-03,NNA-08-05,NIS-07-08,NIS-08-04,NDM-06-04,NDM-07-05
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές...",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
ects = pd.read_csv("subjects_ects.csv")
ects_dict = ects.set_index('Subjects')['Ects'].to_dict()
#display(ects_dict)

In [ ]:
#data.describe()
#data.info()

## Engineering ML Features (GPA and ECTS)

In [ ]:
data_courses = data.iloc[:,10:]
ects_total = 0
total = 0
gpa = 0
number = 0
gpa_list = []
ects_list = []
ects_needed = []
number_of_prior_courses = []
for i in range (0, 43):
    for column in data_courses.columns[:]:
        temp = data_courses.at[i,column]
        if temp >= 5:
            number = number + 1
            total = total + temp * ects_dict.get(column)
            ects_total = ects_total + ects_dict.get(column)
    #print('GPA: ', total / ects_total , 'ECTS: ', ects_total)
    number_of_prior_courses.append(number)
    ects_list.append(ects_total)
    ects_needed.append(240-ects_total)
    gpa_list.append(total / ects_total)
    ects_total = 0
    total = 0
    number = 0

In [ ]:
#display(number_of_prior_courses)
#display(ects_needed)
#display(ects_list)
#display(gpa_list)

#data['prior_courses'] = number_of_prior_courses
#data['ects_needed'] = ects_needed
#data['ECTS'] = ects_list
#data['GPA'] = gpa_list
#display(data)

### Rename Columns ###

In [ ]:
# Change Courses Names
import json
from pandas.io.json import json_normalize

with open("courses_ids_600000014.json","r") as json_file:
     json_file = json.load(json_file)
coded_courses = json_normalize(json_file['courses'])
del coded_courses['ccoursecode']
course_dict = coded_courses.set_index('coursecode')['courseId'].to_dict()
#display(course_dict)

In [ ]:
data.rename(columns={"Ηλικία":"age","Φύλο":"gender","Επέλεξα τη σχολή μου διότι:":"reason",
                     "Κατά μέσο όρο την εβδομάδα, διαβάζω:":"study_time",
                    "Μέσα στο εξάμηνο, παρακαλουθώ:":"lectures",
                     "Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;":"private",
                    "Μετά το πτυχίο, θα ήθελα να ακολουθήσω:":"postgraduate",
                    "Ποιο από τα παρακάτω ισχύει;":"roomates",
                    "Η σχολή απέχει από το σπίτι μου:":"distance",
                    "Ασχολούμαι εβδομαδιαία με:":"hobbies"},inplace=True)
for key in course_dict.keys():
    data.rename(columns = {key:course_dict[key]}, inplace=True)
display(data)

,age,gender,reason,study_time,lectures,private,postgraduate,roomates,distance,hobbies,...,NGE-08-02,NGE-07-03,NGE-06-04,NGE-06-05,NGE-08-03,NNA-08-05,NIS-07-08,NIS-08-04,NDM-06-04,NDM-07-05
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές...",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


### Categorical Values Handling - Function ###

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MultiLabelBinarizer


def handle_categorical(data):
    
    # Let's split to characteristics and courses
    data_characteristics = data.iloc[:,:10]
    data_courses = data.iloc[:,10:]

    
    # Categorical values that maintain the scaling properties "study_time","lectures","postgraduate","distance"
    # Study_time
    data_characteristics["study_time"] = data_characteristics["study_time"].replace({"0 - 2 ώρες":0.0,"2 - 5 ώρες":1.0,"> 5 ώρες":2.0})
    data_characteristics["study_time"] = pd.to_numeric(data_characteristics["study_time"])

    # Lectures
    data_characteristics["lectures"] = data_characteristics["lectures"].replace({"Λιγότερες από τις μισές διαλέξεις":0.0,
                                                 "Περίπου τις μισές διαλέξεις":1.0,
                                                 "Παραπάνω από τις μισές διαλέξεις":2.0,
                                                 "Όλες τις διαλέξεις":3.0})
    data_characteristics["lectures"] = pd.to_numeric(data_characteristics["lectures"])

    # Postgraduate
    data_characteristics["postgraduate"] = data_characteristics["postgraduate"].replace({"Τίποτα από τα δύο":0.0,
                                                         "Μεταπτυχιακές Σπούδες":1.0,
                                                         "Διδακτορικές Σπουδές":2.0})
    data_characteristics["postgraduate"] = pd.to_numeric(data_characteristics["postgraduate"])

    # Distance
    data_characteristics["distance"] = data_characteristics["distance"].replace({"< 10 λεπτά":0.0,
                                                 "10 - 25 λεπτά":1.0,
                                                 "25 - 45 λεπτά":2.0,
                                                 "> 45 λεπτά":3.0})
    data_characteristics["distance"] = pd.to_numeric(data_characteristics["distance"])

    # Gender
    data_characteristics["gender"] = data_characteristics["gender"].replace({"Κορίτσι":1,"Αγόρι":0})
    data_characteristics["gender"] = pd.to_numeric(data_characteristics["gender"])

    # private
    data_characteristics["private"] = data_characteristics["private"].replace({"Ναι":0,
                                              "Όχι":1})
    data_characteristics["private"] = pd.to_numeric(data_characteristics["private"])
    
    
    # One-hot encoder columns (only roomates)

    ohe_columns = ["roomates"]

    full_pipeline = ColumnTransformer([
        ('one_hot',OneHotEncoder(),ohe_columns)
    ])

    roomates = full_pipeline.fit_transform(data_characteristics)
    # Concat with  data_characteristics with roomates
    roomates_df = pd.DataFrame(roomates.toarray(),columns=['family','alone','friend','siblings'],dtype=np.int8)
    data_characteristics_updated = pd.concat([data_characteristics.drop("roomates",axis=1),roomates_df],axis=1)
    
    
    
    # Convert string cell with multiple values to list
    acceptable_hobbies = ["Σειρές / Ταινίες","Αθλητισμό","Video Games","Ξένη γλώσσα","Εθελοντισμός"]
    for student in range(0,data_characteristics.shape[0]):
        data_characteristics['reason'][student] = data_characteristics['reason'][student].split(", ") #There is a space after each comma
        # Hobbies transformation

        hobbies_list = data_characteristics['hobbies'][student].split(", ")
        for i in range(0,len(hobbies_list)):
            if hobbies_list[i] not in acceptable_hobbies:
                hobbies_list[i] = "Άλλο"
        data_characteristics['hobbies'][student] = hobbies_list
        
        
        # Multilabel Binarizer

        mlb = MultiLabelBinarizer()
        hobbies = mlb.fit_transform(data_characteristics['hobbies'])
#         print(mlb.classes_)
        reasons = mlb.fit_transform(data_characteristics['reason'])
#         print(mlb.classes_)

    # Convert to data_characteristics frame and concat
    hobbies_df = pd.DataFrame(hobbies,columns=["vgames","other","sports","volunteer","languange","movies"])
    data_characteristics_updated = pd.concat([data_characteristics_updated.drop(["hobbies"],axis=1),hobbies_df],axis=1)

    reasons_df = pd.DataFrame(reasons,columns=["quality","choice","subject","parents","career"])
    data_characteristics_updated = pd.concat([data_characteristics_updated.drop(["reason"],axis=1),reasons_df],axis=1)
    
    full_data = pd.concat([data_characteristics_updated,data_courses],axis=1)
    
    return full_data, data_characteristics_updated.columns, data_courses.columns



### Handle Categorical ###

In [6]:
data_updated, characteristics_cols, courses_cols = handle_categorical(data)
display(characteristics_cols)
display(courses_cols)

Index(['age', 'gender', 'study_time', 'lectures', 'private', 'postgraduate',
       'distance', 'family', 'alone', 'friend', 'siblings', 'vgames', 'other',
       'sports', 'volunteer', 'languange', 'movies', 'quality', 'choice',
       'subject', 'parents', 'career'],
      dtype='object')

Index(['NCO-01-01', 'NCO-01-02', 'NCO-01-03', 'NCO-01-04', 'NCO-01-05',
       'NCO-02-01', 'NNA-06-01', 'NNA-06-02', 'NNA-07-01', 'NCO-02-02',
       'NNA-07-02', 'NCO-02-03', 'NCO-02-04', 'NCO-02-05', 'NNA-07-03',
       'NNA-07-04', 'NNA-07-05', 'NNA-07-06', 'NNA-07-08', 'NCO-03-01',
       'NCO-03-02', 'NCO-03-03', 'NCO-03-04', 'NCO-03-05', 'NCO-04-01',
       'NCO-04-02', 'NCO-04-03', 'NCO-04-04', 'NCO-04-05', 'NCO-05-01',
       'NNA-08-03', 'NCO-05-02', 'NCO-05-03', 'NCO-05-04', 'NCO-05-05',
       'NET-06-01', 'NIS-06-01', 'NET-06-02', 'NET-06-03', 'NIS-06-02',
       'NET-07-01', 'NIS-06-03', 'NET-07-02', 'NET-07-03', 'NIS-06-04',
       'NGE-06-01', 'NGE-06-02', 'NGE-06-03', 'NDM-06-01', 'NIS-06-05',
       'NDM-06-02', 'NIS-06-06', 'NET-08-02', 'NIS-07-01', 'NET-08-01',
       'NIS-07-02', 'NIS-07-04', 'NIS-07-05', 'NDM-07-01', 'NGE-07-01',
       'NIS-07-06', 'NDM-07-02', 'NIS-08-01', 'NDM-07-04', 'NIS-08-02',
       'NDM-08-01', 'NDM-08-02', 'NIS-08-03', 'NDM-08-03', 'NIS-

### Creating dictionary with courses as keys and  number of students that passed the subject as values

In [7]:
new_dict = {new_list: [] for new_list in range(0)}
totalSum = 0
iterations = 0 
for columns in data.columns[10:]:
    temp = data[data[columns]>-1].shape[0]
    if temp!=0:
        totalSum = totalSum + temp
        iterations = iterations + 1
    new_dict[columns] = temp
mean = totalSum/iterations
print(mean)
print(new_dict)

13.34375
{'NCO-01-01': 41, 'NCO-01-02': 42, 'NCO-01-03': 39, 'NCO-01-04': 38, 'NCO-01-05': 37, 'NCO-02-01': 39, 'NNA-06-01': 0, 'NNA-06-02': 1, 'NNA-07-01': 0, 'NCO-02-02': 40, 'NNA-07-02': 0, 'NCO-02-03': 35, 'NCO-02-04': 40, 'NCO-02-05': 39, 'NNA-07-03': 0, 'NNA-07-04': 0, 'NNA-07-05': 0, 'NNA-07-06': 0, 'NNA-07-08': 0, 'NCO-03-01': 32, 'NCO-03-02': 26, 'NCO-03-03': 22, 'NCO-03-04': 31, 'NCO-03-05': 31, 'NCO-04-01': 26, 'NCO-04-02': 35, 'NCO-04-03': 30, 'NCO-04-04': 35, 'NCO-04-05': 31, 'NCO-05-01': 9, 'NNA-08-03': 0, 'NCO-05-02': 11, 'NCO-05-03': 13, 'NCO-05-04': 10, 'NCO-05-05': 11, 'NET-06-01': 0, 'NIS-06-01': 7, 'NET-06-02': 5, 'NET-06-03': 1, 'NIS-06-02': 5, 'NET-07-01': 2, 'NIS-06-03': 3, 'NET-07-02': 1, 'NET-07-03': 0, 'NIS-06-04': 6, 'NGE-06-01': 2, 'NGE-06-02': 4, 'NGE-06-03': 3, 'NDM-06-01': 2, 'NIS-06-05': 3, 'NDM-06-02': 2, 'NIS-06-06': 5, 'NET-08-02': 2, 'NIS-07-01': 5, 'NET-08-01': 0, 'NIS-07-02': 1, 'NIS-07-04': 2, 'NIS-07-05': 4, 'NDM-07-01': 0, 'NGE-07-01': 1, 'NIS-0

In [8]:
#List with only the courses we are going to use for the model
alist = []
for key in new_dict.keys():
    if new_dict[key] > 0:
        alist.append(key)
display(alist)
print("length:",len(alist))

In [ ]:
# Select those course columns
course_columns = alist
print(course_columns)

### Use all courses

In [ ]:
# Selecting all courses
course_columns = courses_cols
print(course_columns)

### Only Courses ###

In [ ]:
# Using only the courses
selected_columns = courses_cols
print(selected_columns)

### Only Characteristics ###

In [ ]:
# Using only the user characteristics
selected_columns = characteristics_cols
print(selected_columns)

### Characteristics + Courses ###


In [ ]:
selected_columns = characteristics_cols.to_list() + courses_cols.to_list()
print(selected_columns)

In [ ]:
# Select only those columns
data_selected = data_updated.loc[:,selected_columns]


In [ ]:
sum_error = 0
sum_squared_error = 0
for course_selected in course_columns: #For each course
    print(course_selected)
    errors = []
    sq_errors = []
    if course_selected in data_selected:
        X = data_selected.drop(course_selected,axis=1,inplace=False)
    else:
        X = data_selected
    y = data.loc[:,course_selected]
    loo = LeaveOneOut()
    xgb = XGBRegressor(objective = 'reg:squarederror')
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
#         print(X_train)
        xgb.fit(X_train,y_train)
        predictions = xgb.predict(X_test)
        errors.append(mean_absolute_error(y_test,predictions))
        sq_errors.append(mean_squared_error(y_test,predictions))
    
    sum_error += np.mean(errors)
    sum_squared_error += np.mean(sq_errors)**(1/2)
    print("MAE:" + str(np.mean(errors)))
    print("RMSE:" + str((np.mean(sq_errors))**(1/2)))

print("Mean MAE:" + str( sum_error / (len(course_columns))))
print("Mean RMSE:" + str( sum_squared_error / (len(course_columns))))


### Corellation Table ###

In [ ]:

corrmat = data_selected.corr(method="pearson")
top_corr_features = corrmat.index
plt.figure(figsize=(35, 35))
g = sns.heatmap(data_selected[top_corr_features].corr(), annot=True, cmap="RdYlGn")
plt.show()

## Persistence ##  

In [ ]:
prediction_course = "NCO-04-05"
data_selected = data_selected[data_selected[prediction_course] > 0]
X = data_selected.drop(prediction_course,axis=1,inplace=False)
y = data_selected.loc[:,prediction_course]

In [ ]:
xgb_model = XGBRegressor(objective = 'reg:squarederror')
xgb_model.fit(X,y)

In [ ]:
xgb_model.get_booster().feature_names

In [ ]:
import pickle

pickle.dump(xgb_model, open("xgb_" + prediction_course +".dat", "wb"))

In [ ]:
json = X.loc[0].to_json("row{}.json".format(0))
json